# Contenido <a id='back'></a>

* [Introducción](#intro)
* [Objetivos](#objetivos)
* [Etapas](#etapas)
* [Etapa 1. Información general del archivo de datos](#info_general)
    * [1.1 Exploración de los datos](#info_general_data)
    * [1.2 Valores ausentes](#missing_values)
    * [1.3 Distribución de los valores ausentes](#missing_values_dist)
    * [1.4 Porcentaje de valores ausentes](#missing_values_percent)
    * [1.5 Tabla de distribución de valores ausentes en base a `'income_type'`](#missing_values_table_income)
    * [1.6 Tabla de distribución de valores ausentes en base a `'education'`](#missing_values_table_education)
    * [1.7 Tabla de distribución de valores ausentes en base a `'family_status'`](#missing_values_table_family)
    * [1.8 Tabla de distribución de valores ausentes en base a `'gender'`](#missing_values_table_gender)
* [Etapa 2. Transformación de los datos](#transform_data)
    * [2.1 Corrección de duplicados implícitos en `'education'`](#correct_values_education)
    * [2.2 Valores duplicados](#drop_duplicates)
    * [2.3 Trabajar con valores ausentes](#missing_values_work)
        * [2.3.1 Creación de Diccionarios](#dictionary)
        * [2.3.2 Creación de grupos de edad](#group_age)
    * [2.4 Restaurar valores ausentes en `'total_income'`](#restore_values_income)
    * [2.5 Restaurar valores ausentes en `'days_employed'`](#restore_values_days)
    * [2.6 Función alternativa para restaurar valores ausentes en `'dob_years'`](#restore_values_function)
    * [2.7 Clasificación de los datos](#clasification_data)
        * [2.7.1 Creación de grupos de própositos de préstamo](#clasification_purpose)
        * [2.7.2 Creación de grupos de niños](#clasification_children)
        * [2.7.3 Creación de grupos de nivel de ingreso](#clasification_income)
* [Etapa 3.Prueba de hipótesis](#hypotheses)
    * [3.1 Hipótesis 1: ¿Existe una correlación entre tener hijos y pagar a tiempo?](#hypotheses_1)
    * [3.2 Hipótesis 2: ¿Existe una correlación entre la situación familiar y el pago a tiempo?](#hypotheses_2)
    * [3.3 Hipótesis 3: ¿Existe una correlación entre el nivel de ingresos y el pago a tiempo?](#hypotheses_3)
    * [3.4 Hipótesis 4: ¿Cómo afecta el propósito del crédito a la tasa de incumplimiento?](#hypotheses_4)
* [Conclusión general](#conclusion)

# Introducción <a id='intro'></a>
En este proyecto, estudiaremos la solvencia crediticia de los clientes para conocer que factores pueden ocasionar un incumplimiento de pago de un préstamo. En concreto, averiguaremos si el estado civil y el número de hijos o la cantidad de ingresos de un cliente son factores que influyen en el incumplimiento de pago.

# Objetivos: <a id='objetivos'></a>

Probaremos las siguientes hipótesis:

    1. ¿El estado civil de un cliente influye en la capacidad de pago de un préstamo?
    2. ¿Una mayor cantidad de hijos influye negativamente en la capacidad de pago de un préstamo?
    3. ¿Un cliente con altos ingresos tiene mayor capacidad de pago a tiempo de un préstamo?
    4. ¿El propósito de un préstamo afecta el pago a tiempo de un préstamo?

# Etapas <a id='etapas'></a>
El proyecto consistirá de tres etapas:

    1. Abrir el archivo de datos y mirar la información general.
    2. Exploración de datos y preprocesamiento.
    3. Prueba de hipótesis.

[Contenido](#back)
<a id='info_general'></a>
## Etapa 1. Información general del archivo de datos

In [1]:
# Cargamos todas las librerías
import pandas as pd
from sklearn.impute import KNNImputer
import math

In [2]:
# Cargamos los datos
datos_credito=pd.read_csv('credit_scoring_eng.csv')

In [3]:
# mostramos las columnas del DataFrame
datos_credito.columns

Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')

<a id='info_general_data'></a>
### Exploración de los datos
**Descripción de los datos**
- `children` - el número de hijos en la familia 
- `days_employed` - experiencia laboral en días
- `dob_years` - la edad del cliente en años
- `education` - la educación del cliente
- `education_id` - identificador de educación
- `family_status` - estado civil
- `family_status_id` - identificador de estado civil
- `gender` - género del cliente
- `income_type` - tipo de empleo
- `debt` - ¿había alguna deuda en el pago de un préstamo?
- `total_income` - ingreso mensual
- `purpose` - el propósito de obtener un préstamo

In [4]:
# Veamos cuántas filas y columnas tiene nuestro conjunto de datos
datos_credito.shape

(21525, 12)

In [5]:
# Vamos a mostrar las primeras filas
datos_credito.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding


Observamos que **hay 21,525 observaciones y 12 columnas**. Los nombres de las columnas parecen estar bien definidos. La **información** que muestran varias columnas **no es consistente en formato y tipo de datos**.  **Hay que analizar el tipo de datos que tiene cada columna y si hay valores faltantes o filas duplicadas**. Así cómo **buscar** si las **columnas con** valores categóricos tienen **duplicados implicitos.**

In [6]:
# Obtenemos información general de los datos
datos_credito.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


[Contenido](#back)
<a id='missing_values'></a>
### Valores ausentes

In [7]:
# buscamos la cantidad de valores faltantes en cada columna
datos_credito.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Al parecer en las columnas `'days_employed'` y `'total_income'` hay valores faltantes. Cada una con **2,174 valores faltantes.**

In [8]:
# Veamos la tabla filtrada con valores ausentes de la primera columna donde faltan datos
datos_credito_na=datos_credito[datos_credito['days_employed'].isna()]
datos_credito_na.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding


[Contenido](#back)
<a id='missing_values_dist'></a>
### Distribución de los valores ausentes

Necesitamos ahora poder observar como se distribuyen los valores ausentes de las columnas `'days_employed'` y `'total_income'`con respecto a la edad, escolaridad y estado civil del solicitante.

In [9]:
# Apliquemos agrupación por 'education' para ver la distribución de los valores faltantes.
datos_credito_na_agrupado=datos_credito_na['education'].value_counts()
datos_credito_na_agrupado

secondary education    1408
bachelor's degree       496
SECONDARY EDUCATION      67
Secondary Education      65
some college             55
Bachelor's Degree        25
BACHELOR'S DEGREE        23
primary education        19
SOME COLLEGE              7
Some College              7
PRIMARY EDUCATION         1
Primary Education         1
Name: education, dtype: int64

In [10]:
# Apliquemos agrupación por 'family_status' para ver la distribución de los valores faltantes.
datos_credito_na_agrupado=datos_credito_na['family_status'].value_counts()
datos_credito_na_agrupado

married              1237
civil partnership     442
unmarried             288
divorced              112
widow / widower        95
Name: family_status, dtype: int64

**[Tabla agrupada por `'income_type'`]** <a id='info_tabla'></a>

In [11]:
# Apliquemos agrupación por 'income_type' para ver la distribución de los valores faltantes.
datos_credito_na_agrupado=datos_credito_na['income_type'].value_counts()
datos_credito_na_agrupado

employee         1105
business          508
retiree           413
civil servant     147
entrepreneur        1
Name: income_type, dtype: int64

In [12]:
# Apliquemos agrupación por 'gender' para ver la distribución de los valores faltantes.
datos_credito_na_agrupado=datos_credito_na['gender'].value_counts()
datos_credito_na_agrupado

F    1484
M     690
Name: gender, dtype: int64

[Contenido](#back)
<a id='missing_values_percent'></a>
### Porcentaje de valores ausentes

In [13]:
# Calculemos el porcentaje de valores faltantes respecto del total
m=datos_credito_na['days_employed'].isna().sum()
print(f'La cantidad de datos faltantes para la variable \'days_employed\' es: {m}')
n=datos_credito_na['total_income'].isna().sum()
print(f'La cantidad de datos faltantes para la variable \'total_income\' es: {n}')
N=datos_credito.shape[0]
print(f'La cantidad de datos totales es: {N}')
print(f'El porcentaje de valores faltantes respecto del total es de:{n/N: .0%}')

La cantidad de datos faltantes para la variable 'days_employed' es: 2174
La cantidad de datos faltantes para la variable 'total_income' es: 2174
La cantidad de datos totales es: 21525
El porcentaje de valores faltantes respecto del total es de: 10%


**Conclusión intermedia**

El número total de valores faltantes para la columna `'days_employed'` y la columna `'total_income'` es el mismo en este conjunto de datos por lo que podemos concluir que si hay un valor faltante en la columna `'days_employed'` también lo habrá en la columna `'total_income'`. Además, **hay un 10% de valores faltantes respecto del total de datos,** por lo que quizás sea conveniente tratar de reemplazar esos valores faltantes.

Notamos también que **la variable `'education'` tiene duplicados implícitos** que hay que reemplazar.

Consideremos tres **alternativas para tratar con los valores faltantes:**

    * Omitir las variables con datos faltantes.
    * Omitir los registros donde haya datos faltantes.
    * Estimar los datos faltantes donde podemos reemplazarlos con valores predichos usando los datos presentes.
     
**Podremos considerar que la pérdida es aleatoria si afecta por igual a todas las categorías**, o en caso contrario, puede considerarse que los valores ausentes introducen sesgos que puedan invalidar los resultados.

In [14]:
# Vamos a investigar a los clientes que no tienen datos sobre la característica identificada y la columna con los valores ausentes
datos_credito_na=datos_credito[(datos_credito['days_employed'].isna())&(datos_credito['total_income'].isna())]
datos_credito_na.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

**Posibles razones por las que hay valores ausentes en los datos**

Si observamos la tabla agrupada por `'income_type'` ([Tabla](#info_tabla)) para los datos faltantes, podemos notar que la mayoría de los datos faltantes corresponden a personas retiradas, negociantes y empleados. **Esto pueda deberse a que mucha gente no desea proporcionar su información completa con la esperanza que eso ayude a no negarles un crédito.**

[Contenido](#back)

**Vamos a crear una función que nos permite crear una tabla de distribución de valores ausentes agrupada respecto a otra variable.**

In [15]:
def tabla_distribucion(data,variable_agrupar,variable_nula):
    """Esta función calcula la tabla de distribucion de valores ausentes agrupados por la variable_agrupar
    y con valores ausentes en la variable_nula"""
    # creamos un dataframe con valores 'NaN'
    data_na=data[data[variable_nula].isna()]
    
    # creamos la tabla de distribucion para data_na
    data_na_agrupado=data_na[variable_agrupar].value_counts()
    columnas=data_na_agrupado.index
    datos=data_na_agrupado.values
    tabla_datos_credito_na_agrupado=pd.DataFrame(data=datos,index=columnas)
    tabla_datos_credito_na_agrupado=tabla_datos_credito_na_agrupado.reset_index()
    tabla_datos_credito_na_agrupado.set_axis([variable_agrupar,'count_na'],axis='columns',inplace=True)
    
    # creando la tabla de distribución para el conjunto de datos original
    data_agrupado=data[variable_agrupar].value_counts()
    columnas=data_agrupado.index
    datos=data_agrupado.values
    tabla_datos_credito_agrupado=pd.DataFrame(data=datos,index=columnas)
    tabla_datos_credito_agrupado=tabla_datos_credito_agrupado.reset_index()
    tabla_datos_credito_agrupado.set_axis([variable_agrupar,'count'],axis='columns',inplace=True)
    
    # Comparación de la distribución con respecto a 'variable' entre los datos originales y el conjunto de datos faltantes
    tabla_dist=tabla_datos_credito_agrupado.merge(tabla_datos_credito_na_agrupado,on=variable_agrupar,how='left')
    tabla_dist['percent']=(tabla_dist['count_na']/tabla_dist['count'])*100
    
    # guardamos la tabla de distribución creada
    return tabla_dist

[Contenido](#back)
<a id='missing_values_table_income'></a>
### Tabla de distribución de valores ausentes en base a `'income_type'`

In [16]:
tabla_distribucion(data=datos_credito,variable_agrupar='income_type',variable_nula='days_employed')

,income_type,count,count_na,percent
0,employee,11119,1105.0,9.937944
1,business,5085,508.0,9.990167
2,retiree,3856,413.0,10.710581
3,civil servant,1459,147.0,10.075394
4,entrepreneur,2,1.0,50.000000
5,unemployed,2,NaN,NaN
6,paternity / maternity leave,1,NaN,NaN
7,student,1,NaN,NaN


**Encontremos las distribución de los valores ausentes para 'education', 'family_status' y 'gender'.**

[Contenido](#back)
<a id='missing_values_table_education'></a>
### Tabla de distribución de valores ausentes en base a `'education'`

In [17]:
tabla_distribucion(data=datos_credito,variable_agrupar='education',variable_nula='days_employed')

,education,count,count_na,percent
0,secondary education,13750,1408.0,10.240000
1,bachelor's degree,4718,496.0,10.512929
2,SECONDARY EDUCATION,772,67.0,8.678756
3,Secondary Education,711,65.0,9.142053
4,some college,668,55.0,8.233533
5,BACHELOR'S DEGREE,274,23.0,8.394161
6,Bachelor's Degree,268,25.0,9.328358
7,primary education,250,19.0,7.600000
8,Some College,47,7.0,14.893617
9,SOME COLLEGE,29,7.0,24.137931


[Contenido](#back)
<a id='missing_values_table_family'></a>
### Tabla de distribución de valores ausentes en base a `'family_status'`

In [18]:
tabla_distribucion(data=datos_credito,variable_agrupar='family_status',variable_nula='days_employed')

,family_status,count,count_na,percent
0,married,12380,1237,9.991922
1,civil partnership,4177,442,10.581757
2,unmarried,2813,288,10.238180
3,divorced,1195,112,9.372385
4,widow / widower,960,95,9.895833


[Contenido](#back)
<a id='missing_values_table_gender'></a>
### Tabla de distribución de valores ausentes en base a `'gender'`

In [19]:
tabla_distribucion(data=datos_credito,variable_agrupar='gender',variable_nula='days_employed')

,gender,count,count_na,percent
0,F,14236,1484.0,10.424276
1,M,7288,690.0,9.467618
2,XNA,1,NaN,NaN


**Conclusión intermedia**

Al parecer la distribución de los valores faltantes en la columna `'days_employed'`, agrupando el conjunto de datos por la columna `'income_type'`, así como usando otras columnas, es similar para cada una de las características (aproximadamente un 10%) pero eso no arroja alguna luz sobre el motivo de la presencia de valores faltantes. También **es necesario hacer una búsqueda de valores atípicos o erróneos.**

In [20]:
# Comprobar otras razones y patrones que podrían llevar a valores ausentes
datos_credito.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,19351.000000
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,26787.568355
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,16475.450632
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,3306.762000
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,16488.504500
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,23202.870000
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,32549.611000
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,362496.645000


In [21]:
# 60 años de experiencia laboral equivale aproxidamente a 21,900 días
datos_credito[datos_credito['days_employed']>21900].head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
18,0,400281.136913,53,secondary education,1,widow / widower,2,F,retiree,0,9091.804,buying a second-hand car
24,1,338551.952911,57,secondary education,1,unmarried,4,F,retiree,0,46487.558,transactions with commercial real estate
25,0,363548.489348,67,secondary education,1,married,0,M,retiree,0,8818.041,buy real estate
30,1,335581.668515,62,secondary education,1,married,0,F,retiree,0,27432.971,transactions with commercial real estate


In [22]:
datos_credito[datos_credito['dob_years']==0].head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
99,0,346541.618895,0,Secondary Education,1,married,0,F,retiree,0,11406.644,car
149,0,-2664.273168,0,secondary education,1,divorced,3,F,employee,0,11228.230,housing transactions
270,3,-1872.663186,0,secondary education,1,married,0,F,employee,0,16346.633,housing renovation
578,0,397856.565013,0,secondary education,1,married,0,F,retiree,0,15619.310,construction of own property
1040,0,-1158.029561,0,bachelor's degree,0,divorced,3,F,business,0,48639.062,to own a car


[Contenido](#back)

**Conclusión intermedia**

Hasta el momento, podemos decir que **los valores ausentes afectan por igual a las categorías presentes en la variable `'income_type'`.**

De la tabla anterior de resumen de estadísticas podemos observar que la variable `'days_employed'` presenta **valores atípicos**; algunos **negativos**(cosa que no puede ser) **o muy grandes** (401755.400475) **que implican una experiencia laboral en más de 110 años**. Así mismo, la variable `'dob_years'` también presenta valores problematicos, **edades con valores igual a cero.**

**Conclusiones**

**Los valores ausentes** en las columnas `'days_employed'`y `'total_income'`**pueden tener repercusión en la determinación de la puntuación del crédito,** así que habrá que encontrar la mejor manera de lidiar con esos valores. Muy posiblemente haya que hacer una estimación de esos valores ausentes y quizás la mejor manera de esa hacer las estimaciones de esos valores sea realizar agrupaciones sobre algunas variables, que nos ayuden a obtener una estimación de la media o mediana, en el caso de valores ausentes de tipo númerico, o de una frecuencia, en el caso de una variable categórica.

Dado que la columna `'days_employed'` es la que presenta mayor cantidad de problemas, por el momento la ignoraremos y nos concentraremos en trabajar con las restantes.

[Contenido](#back)
<a id='transform_data'></a>
## Etapa 2. Transformación de los datos

Analizaremos cada columna para ver que correciones tenemos que hacer en cada una de las que presentan problemas. Abordaremos valores duplicados, ausentes, problemáticos y atípicos.

<a id='correct_values_education'></a>
### Corrección de duplicados implicitos en `'education'`

Antes de proceder con otros análisis, vamos a corregir la presencia de duplicados implicitos en la columna `'education'` ya que es la única variable categórica con este tipo de problema.

In [23]:
# Veamos todos los valores en la columna 'education' para verificar si será necesario corregir la ortografía y qué habrá que corregir exactamente
datos_credito['education'].sort_values().unique()

array(["BACHELOR'S DEGREE", "Bachelor's Degree", 'GRADUATE DEGREE',
       'Graduate Degree', 'PRIMARY EDUCATION', 'Primary Education',
       'SECONDARY EDUCATION', 'SOME COLLEGE', 'Secondary Education',
       'Some College', "bachelor's degree", 'graduate degree',
       'primary education', 'secondary education', 'some college'],
      dtype=object)

In [24]:
# Realizaremos una transformación a minúsculas para buscar solucionar el problema
datos_credito['education']=datos_credito['education'].str.lower()

In [25]:
# Comprobamos todos los valores en la columna para asegurarnos de que los hayamos corregido
datos_credito['education'].sort_values().unique()

array(["bachelor's degree", 'graduate degree', 'primary education',
       'secondary education', 'some college'], dtype=object)

In [ ]:
# Reemplazaremos los espacios que existen en los nombres de las categorías por '_' para hacer más fácil su consulta
datos_credito['education'].replace("bachelor's degree","bachelor's_degree",inplace=True)
datos_credito['education'].replace('graduate degree','graduate_degree',inplace=True)
datos_credito['education'].replace('primary education','primary_education',inplace=True)
datos_credito['education'].replace('secondary education','secondary_education',inplace=True)
datos_credito['education'].replace('some college','some_college',inplace=True)
datos_credito['education'].sort_values().unique()

array(["bachelor's_degree", 'graduate_degree', 'primary_education',
       'secondary_education', 'some_college'], dtype=object)

**Analizaremos ahora la columna `'children'`.**

In [26]:
# Veamos la distribución de los valores en la columna 'children'
datos_credito['children'].sort_values().unique()

array([-1,  0,  1,  2,  3,  4,  5, 20])

In [27]:
datos_credito['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Podemos notar que **los valores: -1 y 20 son valores fuera de lugar**. debemos elegir reemplazarlos por algún valor coherente o eliminarlos si no se encuentra una mejor solución. Tenemos un total de 123 datos problemáticos de un total de 21525, lo cual representa un 0.6%. Asumiendo que posiblemente hubo errores en su captura,procedemos a reeemplazar los valores -1 por 1 y 20 por 2.

[Contenido](#back)

In [28]:
# Eliminaremos los datos donde la columna 'children' sea igual a -1 ó 20
datos_credito['children'].replace(-1,1,inplace=True)
datos_credito['children'].replace(20,2,inplace=True)

In [29]:
# Comprobar la columna 'children' de nuevo para asegurarnos de que todo está arreglado
datos_credito['children'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

Procedemos ahora a analizar  la columna `'days_employed'`, ya que contiene datos faltantes y problemáticos.

In [30]:
# Encontramos los datos problemáticos en 'days_employed', y calculamos el porcentaje
print(datos_credito.isna().sum())
datos_credito['days_employed'].describe()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64


count     19351.000000
mean      63046.497661
std      140827.311974
min      -18388.949901
25%       -2747.423625
50%       -1203.369529
75%        -291.095954
max      401755.400475
Name: days_employed, dtype: float64

**Podemos notar que hay 2,174 valores faltantes en la columna `'days_employed'`** y esto es un número alto **(10% del total)**. Hay que tratar de encontrar solución a este problema. Además, tenemos valores negativos y algunos atípicos. Asumiremos que los valores negativos son causa de error al ingresar los datos y los cambiaremos a positivos.

**Los valores atípicos asumiremos que son a causa de una mala escala en los datos y que se proporcionó la información en horas, por lo que para llevar los valores a días laborables dividiremos las columna entre 24, que son las horas correpondientes a un día.**

In [31]:
def replace_valor(row):
    """Esta función reemplaza los valores negativos por positivos en la columna 'days_employed' y los divide entre 24 horas
    row= el renglón en el conjunto de datos donde se va a realizar el reemplazo
    dias_empleo= los dias de experiencia laboral del individuo
    """

    #Obtenemos el valor de tipo_empleo
    dias_empleo=row['days_employed']
    if dias_empleo<0:
        valor=dias_empleo*(-1)/24
        return valor
    else:
        valor=dias_empleo/24
        return valor

Reemplazamos los valores negativos en esta columna por valores positivos y hacemos el cambio de escala.

In [32]:
# aplicamos la función 'replace_valor'
datos_credito['days_employed']=datos_credito.apply(replace_valor,axis=1)
datos_credito.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,351.569709,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,167.700156,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,234.309275,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,171.864467,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,14177.753002,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding


In [33]:
# Nos aseguramos que se haya hecho el reemplazo
datos_credito.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,19351.000000
mean,0.479721,2788.113704,43.293380,0.817236,0.972544,0.080883,26787.568355
std,0.755528,5792.953355,12.574584,0.548138,1.420324,0.272661,16475.450632
min,0.000000,1.005901,0.000000,0.000000,0.000000,0.000000,3306.762000
25%,0.000000,38.625386,33.000000,1.000000,0.000000,0.000000,16488.504500
50%,0.000000,91.425857,42.000000,1.000000,0.000000,0.000000,23202.870000
75%,1.000000,230.745102,53.000000,1.000000,1.000000,0.000000,32549.611000
max,5.000000,16739.808353,75.000000,4.000000,4.000000,1.000000,362496.645000


[Contenido](#back)

Ahora buscamos algún problema a la edad del cliente `'dob_years'`.

In [34]:
# Revisaremos la columna `dob_years` en busca de valores sospechosos
datos_credito['dob_years'].value_counts()

35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

Observamos que en este nuevo conjunto de datos hay 101 personas con edad igual a cero, valores no creíbles en la columna `'dob_years'`.

In [35]:
# Por el momento reemplazaremos los registros con edades iguales a cero por valores 'NaN' para después estimarlos
datos_credito.loc[datos_credito['dob_years']==0,'dob_years']=float('nan')
datos_credito['dob_years'].isna().sum()

101

In [36]:
# Comprobamos el resultado creando un DataFrame con las edades con valor a cero
datos_credito_filter_years=datos_credito[datos_credito['dob_years']==0]
datos_credito_filter_years

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


**Observamos que el DataFrame creado está vacío**, lo cual nos indica que los registros con edades igual a cero se han reemplazado correctamente.

Revisaremos ahora la columna `'family_status'`en búsqueda de problemas.

In [37]:
# Veamos los valores de la columna 'family_status'
datos_credito['family_status'].value_counts()

married              12380
civil partnership     4177
unmarried             2813
divorced              1195
widow / widower        960
Name: family_status, dtype: int64

In [ ]:
# Reemplazaremos los espacios que existen en los nombres de las categorías por '_' para hacer más fácil su consulta
datos_credito['family_status'].replace('civil partnership','civil_partnership',inplace=True)
datos_credito['family_status'].replace('widow / widower','widow_widower',inplace=True)
datos_credito['family_status'].sort_values().unique()

array(['civil_partnership', 'divorced', 'married', 'unmarried',
       'widow_widower'], dtype=object)

Revisaremos la columna `'gender'` para ver qué tipo de valores hay y qué problemas puede tener.

In [38]:
# Veamos los valores en la columna
datos_credito['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

***Reemplazaremos la categoría 'XNA' que es un solo valor,*** por el valor más probable que en este caso es **F**.

In [39]:
# Nos aseguramos de que esté arreglado
datos_credito.loc[datos_credito['gender']=='XNA','gender']='F'
datos_credito['gender'].value_counts()

F    14237
M     7288
Name: gender, dtype: int64

Podemos notar que hay sólo dos categorías en la columna `'gender'`; F y M. El registro correspondiente al género 'XNA' se reeemplazó correctamente.

Revisaremos la columna `'income_type'`, para ver qué problemas puede tener.

In [40]:
# Veamos los valores en la columna 'income_type'
datos_credito['income_type'].value_counts()

employee                       11119
business                        5085
retiree                         3856
civil servant                   1459
entrepreneur                       2
unemployed                         2
paternity / maternity leave        1
student                            1
Name: income_type, dtype: int64

In [ ]:
# Reemplazaremos los espacios que existen en los nombres de las categorías por '_' para hacer más fácil su consulta
datos_credito['income_type'].replace('civil servant','civil_servant',inplace=True)
datos_credito['income_type'].replace('paternity / maternity leave','paternity_maternity_leave',inplace=True)
datos_credito['income_type'].sort_values().unique()

array(['business', 'civil_servant', 'employee', 'entrepreneur',
       'paternity_maternity_leave', 'retiree', 'student', 'unemployed'],
      dtype=object)

[Contenido](#back)
<a id='drop_duplicates'></a>
### Valores duplicados

In [41]:
# Buscamos valores duplicados
datos_credito.duplicated().sum()

71

In [42]:
# Procedemos a eliminar los valores duplicados
datos_credito=datos_credito.drop_duplicates().reset_index(drop=True)

In [43]:
# Última comprobación para ver si tenemos duplicados
datos_credito.duplicated().sum()

0

In [44]:
# Comprobamos el tamaño del conjunto de datos resultante de eliminar filas duplicadas
datos_credito.shape

(21454, 12)

Originalmente teníamos **N=21,525** observaciones, en este nuevo conjunto de datos donde se han quitado valores problemáticos y duplicados tenemos **n=21,454**, lo que nos implica que nos hemos quedado con un 99.6% de la cantidad de datos que teníamos en un inicio. Aún tenemos valores ausentes en varias columnas.

[Contenido](#back)
<a id='missing_values_work'></a>
### Trabajar con valores ausentes

Crearemos dos diccionarios: uno para la variable **'education'** y otro para **'family_status'**.

<a id='dictionary'></a>
#### Creación de Diccionarios

In [45]:
# Creamos los diccionarios
datos_credito.head(1)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,351.569709,42.0,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house


In [46]:
lista=datos_credito.groupby(by=['education','education_id'])['education'].count().index
lista

MultiIndex([(  'bachelor's degree', 0),
            (    'graduate degree', 4),
            (  'primary education', 3),
            ('secondary education', 1),
            (       'some college', 2)],
           names=['education', 'education_id'])

In [47]:
# creamos el diccionario de datos para 'education'
education_dic={}
for i in range(len(lista)):
    key,value=lista[i]
    education_dic[key]=value

In [48]:
education_dic

{"bachelor's degree": 0,
 'graduate degree': 4,
 'primary education': 3,
 'secondary education': 1,
 'some college': 2}

In [49]:
lista=datos_credito.groupby(by=['family_status','family_status_id'])['family_status'].count().index
lista

MultiIndex([('civil partnership', 1),
            (         'divorced', 3),
            (          'married', 0),
            (        'unmarried', 4),
            (  'widow / widower', 2)],
           names=['family_status', 'family_status_id'])

In [50]:
# creamos el diccionario de datos para 'family_status'
family_dic={}
for i in range(len(lista)):
    key,value=lista[i]
    family_dic[key]=value

In [51]:
family_dic

{'civil partnership': 1,
 'divorced': 3,
 'married': 0,
 'unmarried': 4,
 'widow / widower': 2}

[Contenido](#back)
<a id='group_age'></a>
#### Creación de grupos de edad

Las columnas `'days_employed'` y `'total_income'` tienen valores ausentes en el **DataFrame 'datos_credito'**.

Empezaremos por crear grupos de categorías a diversas variables para poder hacer estimaciones de los valores ausentes.

Por el momento vamos a trabajar con el **DataDrame 'datos_credito'**.

In [52]:
# Vamos a escribir una función que agrupe la edad en categorias
def age_group(age):
    """La función regresa el grupo de edad de acuerdo al valor de la edad,
    usando las siguientes reglas:
    -'menor de 30' para age<=30
    -'31-40' para 30<age<=40
    -'41-50' para 40<age<=50
    -'51-60' para 50<age<=60
    -'61-70' para 60<age<=70
    -'70+' para age>70
    """
    if age<=30:
        return 'menor de 30'
    elif age<=40:
        return '31-40'
    elif age<=50:
        return '41-50'
    elif age<=60:
        return '51-60'
    elif age<=70:
        return '61-70'
    else:
        return '70+'

In [53]:
# Probamos la función
edades=[15,30,36,46,86]
res=list(map(age_group,edades))
res

['menor de 30', 'menor de 30', '31-40', '41-50', '70+']

In [54]:
# Creamos una nueva columna basada en la función
datos_credito['age_group']=datos_credito['dob_years'].apply(age_group)

In [55]:
# Comprobamos los valores en la nueva columna
datos_credito.reset_index(drop=True)
datos_credito.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
0,1,351.569709,42.0,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,41-50
1,1,167.700156,36.0,secondary education,1,married,0,F,employee,0,17932.802,car purchase,31-40
2,0,234.309275,33.0,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,31-40
3,3,171.864467,32.0,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,31-40
4,0,14177.753002,53.0,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,51-60
5,0,38.591076,27.0,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house,menor de 30
6,0,119.966752,43.0,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions,41-50
7,0,6.365815,50.0,secondary education,1,married,0,M,employee,0,21731.829,education,41-50
8,2,288.744387,35.0,bachelor's degree,0,civil partnership,1,F,employee,0,15337.093,having a wedding,31-40
9,0,91.198185,41.0,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family,41-50


Generalmente, los ingresos de una persona están más relacionados con el tipo de empleo y el nivel de educación, y menos relacionados con sólo la edad de la persona.

[Contenido](#back)

In [56]:
# Observamos los valores ausentes en nuestro último DataFrame creado
print("La dimensión del DataFrame es:",datos_credito.shape)
datos_credito.isna().sum()

La dimensión del DataFrame es: (21454, 13)


children               0
days_employed       2103
dob_years            101
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2103
purpose                0
age_group              0
dtype: int64

A partir de este último DataFrame **'datos_credito'**, vamos a **crear un DataFrame sin valores ausentes para estimar la media y la mediana.**

In [57]:
# Creamos un DataFrame sin valores ausentes
datos_credito_filter_notna=datos_credito[datos_credito['total_income'].notna()]
datos_credito_filter_notna.isna().sum()

children             0
days_employed        0
dob_years           91
education            0
education_id         0
family_status        0
family_status_id     0
gender               0
income_type          0
debt                 0
total_income         0
purpose              0
age_group            0
dtype: int64

In [58]:
# Examinamos los valores medios y medianos de los ingresos en función de 'income_type'
datos_credito_filter_notna.pivot_table(index='income_type',values='total_income',aggfunc=['mean','median'])

,mean,median
,total_income,total_income
income_type,,
business,32386.793835,27577.2720
civil servant,27343.729582,24071.6695
employee,25820.841683,22815.1035
entrepreneur,79866.103000,79866.1030
paternity / maternity leave,8612.661000,8612.6610
retiree,21940.394503,18962.3180
student,15712.260000,15712.2600
unemployed,21014.360500,21014.3605


In [59]:
# Examinamos los valores medios y medianos de los ingresos en función de 'education'
datos_credito_filter_notna.pivot_table(index='education',values='total_income',aggfunc=['mean','median'])

,mean,median
,total_income,total_income
education,,
bachelor's degree,33142.802434,28054.5310
graduate degree,27960.024667,25161.5835
primary education,21144.882211,18741.9760
secondary education,24594.503037,21836.5830
some college,29045.443644,25618.4640


In [60]:
# Examinamos los valores medios y medianos de los ingresos en función de 'income_type' y 'education'
datos_credito_filter_notna.pivot_table(index=['income_type','education'],values='total_income',aggfunc=['mean','median'])

mean       median
                                                 total_income total_income
income_type                 education                                     
business                    bachelor's degree    38780.136881   32285.6640
                            primary education    26409.124931   21887.8250
                            secondary education  28718.435242   25451.3100
                            some college         31623.893705   28778.7440
civil servant               bachelor's degree    31571.287664   27601.7775
                            graduate degree      17822.757000   17822.7570
                            primary education    29449.016667   23734.2870
                            secondary education  24648.816597   21864.4750
                            some college         27596.312587   25694.7750
employee                    bachelor's degree    30650.288996   26502.5190
                            graduate degree      31089.653667   31771.3210
                            primary education    21954.056075   20159.1860
                            secondary education  24426.079549   21848.8175
                            some college         27951.531586   24209.4300
entrepreneur                bachelor's degree    79866.103000   79866.1030
paternity / maternity leave secondary education   8612.661000    8612.6610
retiree                     bachelor's degree    27306.878056   23078.5230
                            graduate degree      28334.215000   28334.2150
                            primary education    17810.387914   16415.7850
                            secondary education  21071.829349   18374.8570
                            some college         22129.937314   19221.9030
student                     bachelor's degree    15712.260000   15712.2600
unemployed                  bachelor's degree    32435.602000   32435.6020
                            secondary education   9593.119000    9593.1190

In [61]:
# Examinamos los valores medios y medianos de los ingresos en función de 'age_group'
datos_credito_filter_notna.pivot_table(index='age_group',values='total_income',aggfunc=['mean','median'])

,mean,median
,total_income,total_income
age_group,,
31-40,28376.735148,24825.1865
41-50,28390.207085,24569.9680
51-60,25482.856294,22056.7710
61-70,23245.390243,19705.8550
70+,22348.122524,20384.0430
menor de 30,25817.674826,22957.1850


<a id='nota'></a>
A pesar que podría realizarse una mejor estimación de los valores ausentes en la variable `'total_income'` usando la agrupación por `'income_type'` y `'education'`, realizar la desagregación a niveles tan detallados nos puede llevar a que la cantidad de datos para hacer la estimación de la media o mediana sea baja o que alguna posible combinación de esta desagregación no sea posible estimarla. Por el momento, optaremos por **realizar la estimación usando la agrupación de esta manera.** ....[Continuar](#continuar)

[Contenido](#back)
<a id='restore_values_income'></a>
###  Restaurar valores ausentes en `'total_income'`

Para restaurar los valores ausentes en `'total_income'` crearemos un DataFrame que contenga los valores a usar como reemplazo. Haremos las estimaciones agrupando por `'income_type'`y `'education'` y posteriormente obtendremos su media.

In [62]:
# creamos un DataFrame sin valores NaN para estimar los valores a usar de reemplazo
data_notna=datos_credito[datos_credito['total_income'].notna()]
    
# creamos la tabla de valores estimados
testimados=data_notna.pivot_table(index=['income_type','education'],values='total_income',aggfunc='mean')
indices=testimados.index
datos=testimados.values
df=pd.DataFrame(data=datos,index=indices)
df=df.reset_index()
df.columns
df.set_axis(['income_type','education','media'],axis='columns',inplace=True)

In [63]:
#  creamos una función para hacer el reemplazo de los valores ausentes en 'total_income' en base a 'income_type' y 'education'
def replace_missing_total(row):
    """Esta función reeemplaza los valores 'NaN' en la variable 'total_income' con la media agrupada por
    'income_type' y 'education'
    df= el DataFrame que contiene los valores a usar de reemplazo
    """
   
    # probamos la condicion en cada fila del DataFrame
    ingreso_total=row['total_income']
    
    try:
        if math.isnan(ingreso_total):
            tipo_ingreso=row['income_type']
            educacion=row['education']

            #Obtenemos el valor a usar de reemplazo
            media=df.loc[df['income_type']==tipo_ingreso].loc[df['education']==educacion]['media']
            total_income_dos=float(media)
            return total_income_dos
        else:
            total_income_dos=row['total_income']
            return total_income_dos
    except:
        return float('nan')

In [64]:
#Reemplazamos los valores NaN en 'income_type'
datos_credito['total_income']=datos_credito.apply(replace_missing_total,axis=1)

In [65]:
datos_credito[datos_credito['days_employed'].isna()].head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
12,0,NaN,65.0,secondary education,1,civil partnership,1,M,retiree,0,21071.829349,to have a wedding,61-70
26,0,NaN,41.0,secondary education,1,married,0,M,civil servant,0,24648.816597,education,41-50
29,0,NaN,63.0,secondary education,1,unmarried,4,F,retiree,0,21071.829349,building a real estate,61-70
41,0,NaN,50.0,secondary education,1,married,0,F,civil servant,0,24648.816597,second-hand car purchase,41-50
55,0,NaN,54.0,secondary education,1,civil partnership,1,F,retiree,1,21071.829349,to have a wedding,51-60


In [66]:
datos_credito.isna().sum()

children               0
days_employed       2103
dob_years            101
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
age_group              0
dtype: int64

Notamos que ya no hay valores ausentes en la columna `'total_income'`, el reemplazo se hizo de manera correcta.

[Contenido](#back)
<a id='restore_values_days'></a>
###  Restaurar valores ausentes en `'days_employed'`

La variable `'income_type'`puede ayudarnos a reemplazar los valores ausentes en la columna `'days_employed'`. Ya que esta columna presenta valores atípicos y problemáticos, usaremos la mediana y la agrupación por `'income_type'`. **Usaremos el último DataFrame 'datos_credito'** que ya no contiene valores ausentes en la columna `'total_income'`.

In [67]:
# Creamos un DataFrame sin valores nulos en la columna 'days_employed'
datos_credito_filter_notna_employed=datos_credito[datos_credito['days_employed'].notna()]
datos_credito_filter_notna_employed.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19351 entries, 0 to 21453
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          19351 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         19260 non-null  float64
 3   education         19351 non-null  object 
 4   education_id      19351 non-null  int64  
 5   family_status     19351 non-null  object 
 6   family_status_id  19351 non-null  int64  
 7   gender            19351 non-null  object 
 8   income_type       19351 non-null  object 
 9   debt              19351 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           19351 non-null  object 
 12  age_group         19351 non-null  object 
dtypes: float64(3), int64(4), object(6)
memory usage: 2.1+ MB


In [68]:
# Examinamos los valores medios y medianos de 'days_employed' en función de 'income_type' y 'age_group'
datos_credito_filter_notna_employed.pivot_table(index=['income_type','age_group'],values='days_employed',aggfunc=['mean','median','skew'])

mean        median  \
                                        days_employed days_employed   
income_type                 age_group                                 
business                    31-40           79.175056     63.594383   
                            41-50          104.457546     81.014387   
                            51-60          120.816915     85.491393   
                            61-70          151.836463     98.261492   
                            70+            104.048033     72.564183   
                            menor de 30     50.331447     39.752800   
civil servant               31-40          123.103128    112.913596   
                            41-50          167.951519    143.105663   
                            51-60          206.016430    167.794714   
                            61-70          182.946522    138.268337   
                            70+            196.262551    200.446970   
                            menor de 30     70.703109     61.273939   
employee                    31-40           86.239731     65.349298   
                            41-50          115.757158     82.393567   
                            51-60          136.304569     93.131794   
                            61-70          161.805655    111.211415   
                            70+             86.744198     63.120698   
                            menor de 30     52.652648     43.556793   
entrepreneur                menor de 30     21.702003     21.702003   
paternity / maternity leave 31-40          137.364998    137.364998   
retiree                     31-40        15259.091652  15255.028562   
                            41-50        15228.451258  15271.865433   
                            51-60        15192.522449  15186.517842   
                            61-70        15220.447382  15238.263736   
                            70+          15261.525335  15269.078026   
                            menor de 30  15029.457014  15181.174890   
student                     menor de 30     24.114648     24.114648   
unemployed                  31-40        14063.519451  14063.519451   
                            41-50        16470.951611  16470.951611   

                                                 skew  
                                        days_employed  
income_type                 age_group                  
business                    31-40            1.200588  
                            41-50            1.536116  
                            51-60            1.732915  
                            61-70            1.818415  
                            70+              1.909378  
                            menor de 30      1.020859  
civil servant               31-40            0.589287  
                            41-50            0.612268  
                            51-60            0.848569  
                            61-70            1.093690  
                            70+              0.666394  
                            menor de 30      0.613481  
employee                    31-40            1.218962  
                            41-50            1.383324  
                            51-60            1.537021  
                            61-70            1.535884  
                            70+              2.200676  
                            menor de 30      1.159702  
entrepreneur                menor de 30           NaN  
paternity / maternity leave 31-40                 NaN  
retiree                     31-40            0.001141  
                            41-50            0.009142  
                            51-60            0.046043  
                            61-70           -0.000661  
                            70+              0.019541  
                            menor de 30     -0.260693  
student                     menor de 30           NaN  
unemployed                  31-40                 NaN  
                            41-50       

De la tabla anterior notamos que, en general, el valor promedio es mayor al valor de la mediana así que en este caso optaremos por reeemplazar los valores ausentes por la mediana.

In [69]:
# creamos la tabla de valores estimados
testimados=datos_credito_filter_notna_employed.pivot_table(index=['income_type','age_group'],values='days_employed',aggfunc='median')
indices=testimados.index
datos=testimados.values
df=pd.DataFrame(data=datos,index=indices)
df=df.reset_index()
df.columns
df.set_axis(['income_type','age_group','median'],axis='columns',inplace=True)

In [70]:
# Creamos la función para reemplazar los valores ausentes en 'days_employed'
def replace_missing_values_days_employed(row):
    """Esta función reemplaza los valores ausentes en la columna 'days_employed'
    row= el renglón en el conjunto de datos donde se va a realizar el reemplazo
    df= el DataFrame que contiene los valores estimados para el reemplazo
    tipo_empleo= el tipo de empleo del individuo
    grupo_edad=el grupo de edad a que pertenece
    dias_empleo= los dias de experiencia laboral del individuo
    """
   
    # probamos la condicion en cada fila del DataFrame
    dias_empleo=row['days_employed']
    try:    
        if math.isnan(dias_empleo):
            tipo_empleo=row['income_type']
            grupo_edad=row['age_group']

            #Obtenemos el valor a usar de reemplazo
            mediana=df.loc[df['income_type']==tipo_empleo].loc[df['age_group']==grupo_edad]['median']
            days_employed_dos=float(mediana)
            return days_employed_dos
        else:
            days_employed_dos=row['days_employed']
            return days_employed_dos
    except:
        return float('nan')

In [71]:
# Creamos un conjunto de datos para aplicar la función y ver que funcione
indices=['days_employed','income_type','age_group']
valores=[[-256,"business",'menor de 30'],
        [float('nan'),"employee",'51-60'],
         [2546,"paternity / maternity leave",'31-40'],
         [float('nan'),"retiree",'menor de 30'],
         [float('nan'),"student",'41-50'],
         [2456,"unemployed",'51-60'],
         [float('nan'),"business",'70+'],
         [float('nan'),"civil servant",'31-40']
        ]
datos=pd.DataFrame(valores,columns=indices)
datos

,days_employed,income_type,age_group
0,-256.0,business,menor de 30
1,NaN,employee,51-60
2,2546.0,paternity / maternity leave,31-40
3,NaN,retiree,menor de 30
4,NaN,student,41-50
5,2456.0,unemployed,51-60
6,NaN,business,70+
7,NaN,civil servant,31-40


In [72]:
datos['days_employed']=datos.apply(replace_missing_values_days_employed,axis=1)
datos

,days_employed,income_type,age_group
0,-256.000000,business,menor de 30
1,93.131794,employee,51-60
2,2546.000000,paternity / maternity leave,31-40
3,15181.174890,retiree,menor de 30
4,NaN,student,41-50
5,2456.000000,unemployed,51-60
6,72.564183,business,70+
7,112.913596,civil servant,31-40


In [73]:
# Aplicamos la función a 'days_employed'
datos_credito['days_employed']=datos_credito.apply(replace_missing_values_days_employed,axis=1)

In [74]:
# Verficamos que se haya hecho el reemplazo
datos_credito[datos_credito['days_employed'].isna()].head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
5931,0,NaN,58.0,bachelor's degree,0,married,0,M,entrepreneur,0,79866.103,buy residential real estate,51-60


In [ ]:
# Comprobamos las entradas en todas las columnas: asegurando que hayamos corregido todos los valores ausentes
datos_credito.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21454 non-null  int64  
 1   days_employed     21454 non-null  float64
 2   dob_years         21353 non-null  float64
 3   education         21454 non-null  object 
 4   education_id      21454 non-null  int64  
 5   family_status     21454 non-null  object 
 6   family_status_id  21454 non-null  int64  
 7   gender            21454 non-null  object 
 8   income_type       21454 non-null  object 
 9   debt              21454 non-null  int64  
 10  total_income      21454 non-null  float64
 11  purpose           21454 non-null  object 
 12  age_group         21454 non-null  object 
dtypes: float64(3), int64(4), object(6)
memory usage: 2.1+ MB


<a id='continuar'></a>
**Observamos que tenemos un valor ausente y como se había hecho notar anteriormente ([Nota](#nota)), el uso tan desagregado de varias variables para estimar lo valores ausentes podría ocasionarnos este tipo de problemas.** Vamos a reemplazar este único dato ausente por la mediana de la columna `'days_employed'`.

In [ ]:
# buscamos el valor ausente que aún queda por reemplazar
datos_credito[datos_credito['days_employed'].isna()].head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
5931,0,NaN,58.0,bachelor's_degree,0,married,0,M,entrepreneur,0,79866.103,buy residential real estate,51-60


In [ ]:
# Reemplazamos el último valor ausente en la columna 'days_employed'
datos_credito.loc[datos_credito['days_employed'].isna(),'days_employed']=datos_credito['days_employed'].median()
datos_credito.iloc[5931]

children                                      0
days_employed                         85.491393
dob_years                                  58.0
education                     bachelor's_degree
education_id                                  0
family_status                           married
family_status_id                              0
gender                                        M
income_type                        entrepreneur
debt                                          0
total_income                          79866.103
purpose             buy residential real estate
age_group                                 51-60
Name: 5931, dtype: object

In [ ]:
# Comprobamos las entradas en todas las columnas: asegurando que hayamos corregido todos los valores ausentes
datos_credito.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21454 non-null  int64  
 1   days_employed     21454 non-null  float64
 2   dob_years         21353 non-null  float64
 3   education         21454 non-null  object 
 4   education_id      21454 non-null  int64  
 5   family_status     21454 non-null  object 
 6   family_status_id  21454 non-null  int64  
 7   gender            21454 non-null  object 
 8   income_type       21454 non-null  object 
 9   debt              21454 non-null  int64  
 10  total_income      21454 non-null  float64
 11  purpose           21454 non-null  object 
 12  age_group         21454 non-null  object 
dtypes: float64(3), int64(4), object(6)
memory usage: 2.1+ MB


[Contenido](#back)
<a id='restore_values_function'></a>
###  Función alternativa para restaurar valores ausentes en `'dob_years'`

Para la estimación de los valores ausentes en esta columna vamos a usar la siguiente función basada en la teoría de los **K-neighbors.**

In [76]:
# Vamos a usar el siguiente codigo para completar los valores ausentes

# Construimos el modelo

imputer = KNNImputer(n_neighbors=5, weights="uniform")

# Ajustamos el modelo y estimamos los valores ausentes

imputer.fit(datos_credito[["dob_years"]])

datos_credito["dob_years"] = imputer.transform(datos_credito[["dob_years"]]).ravel()

In [77]:
# Comprobamos los valores ausentes en 'datos_credito'
datos_credito.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21454 non-null  int64  
 1   days_employed     21453 non-null  float64
 2   dob_years         21454 non-null  float64
 3   education         21454 non-null  object 
 4   education_id      21454 non-null  int64  
 5   family_status     21454 non-null  object 
 6   family_status_id  21454 non-null  int64  
 7   gender            21454 non-null  object 
 8   income_type       21454 non-null  object 
 9   debt              21454 non-null  int64  
 10  total_income      21454 non-null  float64
 11  purpose           21454 non-null  object 
 12  age_group         21454 non-null  object 
dtypes: float64(3), int64(4), object(6)
memory usage: 2.1+ MB


[Contenido](#back)
<a id='clasification_data'></a>
### Clasificación de los datos

Para poder responder a las preguntas y probar las diferentes hipótesis, será necesario trabajar con datos clasificados.

Empezaremos por clasificar la variable `'debt'` como:
* 0--- 'Sin deuda'
* 1--- 'Con deuda'

In [78]:
datos_credito['debt'].replace(0,'Sin deuda',inplace=True)
datos_credito['debt'].replace(1,'Con deuda',inplace=True)
print()

In [ ]:
datos_credito['debt'].value_counts()

Sin deuda    19713
Con deuda     1741
Name: debt, dtype: int64

[Contenido](#back)
<a id='clasification_purpose'></a>
#### Creación de grupo de própositos del préstamo

Clasificamos ahora la variable `'purpose'`.

In [80]:
# Vamos a comprobar los valores únicos
datos_credito['purpose'].str.lower().sort_values().unique()

array(['building a property', 'building a real estate',
       'buy commercial real estate', 'buy real estate',
       'buy residential real estate', 'buying a second-hand car',
       'buying my own car', 'buying property for renting out', 'car',
       'car purchase', 'cars', 'construction of own property',
       'education', 'getting an education', 'getting higher education',
       'going to university', 'having a wedding', 'housing',
       'housing renovation', 'housing transactions', 'profile education',
       'property', 'purchase of a car', 'purchase of my own house',
       'purchase of the house', 'purchase of the house for my family',
       'real estate transactions', 'second-hand car purchase',
       'supplementary education', 'to become educated', 'to buy a car',
       'to get a supplementary education', 'to have a wedding',
       'to own a car', 'transactions with commercial real estate',
       'transactions with my real estate', 'university education',
       'we

Podemos identificar diversos própositos con lo que se está pidiendo un préstamo:
* Construcción, compra  o renovación de una casa o propiedad (grupo-inversión inmobiliaria)
* Compra de un carro (grupo-compra carro)
* Prepararse académicamente (grupo- preparación académica)
* Casarse (grupo-casarse)

In [81]:
# Escribimos una función para clasificar los datos en función del propósito del préstamo
def group_purpose(row):
    """Esta función crea grupos de clasificación para el próposito de un préstamo"""
     
    if 'car' in row:
        return 'compra carro'
    elif 'wedding' in row:
        return 'casarse'
    elif ('education' or 'educated' or 'university') in row:
        return 'preparación académica'
    else:
        return 'inversión inmobiliaria'

In [82]:
# Creamos una columna con las categorías y contamos los valores
datos_credito['group_purpose']=datos_credito['purpose'].apply(group_purpose)
datos_credito.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group,group_purpose
0,1,351.569709,42.0,bachelor's degree,0,married,0,F,employee,Sin deuda,40620.102,purchase of the house,41-50,inversión inmobiliaria
1,1,167.700156,36.0,secondary education,1,married,0,F,employee,Sin deuda,17932.802,car purchase,31-40,compra carro
2,0,234.309275,33.0,secondary education,1,married,0,M,employee,Sin deuda,23341.752,purchase of the house,31-40,inversión inmobiliaria
3,3,171.864467,32.0,secondary education,1,married,0,M,employee,Sin deuda,42820.568,supplementary education,31-40,preparación académica
4,0,14177.753002,53.0,secondary education,1,civil partnership,1,F,retiree,Sin deuda,25378.572,to have a wedding,51-60,casarse


In [83]:
# Contamos los valores para la columna 'group_purpose'
datos_credito['group_purpose'].value_counts()

inversión inmobiliaria    11715
compra carro               4306
preparación académica      3109
casarse                    2324
Name: group_purpose, dtype: int64

[Contenido](#back)
<a id='clasification_children'></a>
#### Creación de grupos de niños

Clasificamos ahora la variable `'children'`.

In [84]:
# Obtenemos estadísticas resumidas para la columna 'children'
datos_credito['children'].describe()

count    21454.000000
mean         0.480563
std          0.756069
min          0.000000
25%          0.000000
50%          0.000000
75%          1.000000
max          5.000000
Name: children, dtype: float64

Observamos que hay más personas sin niños, posteriormente podríamos agrupar de uno a dos niños y por último; un tercer grupo con tres a cinco niños.

In [85]:
# Creamos una función para clasificar en diferentes grupos

def group_children(row):
    """Esta función crea grupos de clasificación para la cantidad de hijos"""
     
    if row==0:
        return 'Sin hijos'
    elif row<=2:
        return '2-3 hijos'
    else:
        return '4-5 hijos'

In [86]:
# Creamos una columna con categorías
datos_credito['group_children']=datos_credito['children'].apply(group_children)
datos_credito.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group,group_purpose,group_children
0,1,351.569709,42.0,bachelor's degree,0,married,0,F,employee,Sin deuda,40620.102,purchase of the house,41-50,inversión inmobiliaria,2-3 hijos
1,1,167.700156,36.0,secondary education,1,married,0,F,employee,Sin deuda,17932.802,car purchase,31-40,compra carro,2-3 hijos
2,0,234.309275,33.0,secondary education,1,married,0,M,employee,Sin deuda,23341.752,purchase of the house,31-40,inversión inmobiliaria,Sin hijos
3,3,171.864467,32.0,secondary education,1,married,0,M,employee,Sin deuda,42820.568,supplementary education,31-40,preparación académica,4-5 hijos
4,0,14177.753002,53.0,secondary education,1,civil partnership,1,F,retiree,Sin deuda,25378.572,to have a wedding,51-60,casarse,Sin hijos


In [87]:
# Contamos los valores de cada categoría para ver la distribución
datos_credito['group_children'].value_counts()

Sin hijos    14091
2-3 hijos     6983
4-5 hijos      380
Name: group_children, dtype: int64

[Contenido](#back)
<a id='clasification_income'></a>
#### Creación de grupos de nivel de ingreso

Clasificamos ahora la variable `'total_income'`.

In [88]:
# Obtenemos estadísticas resumidas para la columna 'total_income'
datos_credito['total_income'].describe()

count     21454.000000
mean      26797.900567
std       15724.532575
min        3306.762000
25%       17219.817250
50%       24379.051500
75%       31729.172000
max      362496.645000
Name: total_income, dtype: float64

In [89]:
# Vamos a crear ocho grupos de nivel de ingreso
rango=datos_credito['total_income'].max()-datos_credito['total_income'].min()
int(rango)

359189

Crearemos ocho rangos de nivel de ingreso (NI), con límites de la siguiente manera:
*     0<=N1<=50,000
* 50,000<N2<=100,000
* 100,000<N3<=150,000
* 150,000<N4<=200,000
* 200,000<N5<=250,000
* 250,000<N6<=300,000
* 300,000<N7<=350,000
* 350,000<N8

In [90]:
# Creamos una función para clasificar en base al nivel de ingresos

def group_total_income(row):
    """Esta función crea grupos de clasificación para el nivel de ingreso"""
     
    if row<=50000:
        return "menor a 50,000"
    elif row<=100000:
        return "50,001-100,000"
    elif row<=150000:
        return "100,001-150,000"
    elif row<=200000:
        return "150,001-200,000"
    elif row<=250000:
        return "200,001-250,000"
    elif row<=300000:
        return "250,001-300,000"
    elif row<=350000:
        return "300,001-350,000"
    else:
        return 'mayor de 350,000'

In [91]:
# Creamos una columna con categorías
datos_credito['group_total_income']=datos_credito['total_income'].apply(group_total_income)
datos_credito.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group,group_purpose,group_children,group_total_income
0,1,351.569709,42.0,bachelor's degree,0,married,0,F,employee,Sin deuda,40620.102,purchase of the house,41-50,inversión inmobiliaria,2-3 hijos,"menor a 50,000"
1,1,167.700156,36.0,secondary education,1,married,0,F,employee,Sin deuda,17932.802,car purchase,31-40,compra carro,2-3 hijos,"menor a 50,000"
2,0,234.309275,33.0,secondary education,1,married,0,M,employee,Sin deuda,23341.752,purchase of the house,31-40,inversión inmobiliaria,Sin hijos,"menor a 50,000"
3,3,171.864467,32.0,secondary education,1,married,0,M,employee,Sin deuda,42820.568,supplementary education,31-40,preparación académica,4-5 hijos,"menor a 50,000"
4,0,14177.753002,53.0,secondary education,1,civil partnership,1,F,retiree,Sin deuda,25378.572,to have a wedding,51-60,casarse,Sin hijos,"menor a 50,000"


In [92]:
# Contamos los valores de cada categoría para ver la distribución
datos_credito['group_total_income'].value_counts()

menor a 50,000      20133
50,001-100,000       1222
100,001-150,000        71
150,001-200,000        17
200,001-250,000         5
250,001-300,000         4
mayor de 350,000        2
Name: group_total_income, dtype: int64

[Contenido](#back)
<a id='hypotheses'></a>
## Etapa 3. Prueba de hipótesis

<a id='hypotheses_1'></a>
### ¿Existe una correlación entre tener hijos y pagar a tiempo?

In [93]:
# Veamos el comportamiento de los datos sobre los hijos y los pagos puntuales
tabla_pagos_children=datos_credito.pivot_table(index='group_children',columns='debt',values='total_income',aggfunc='count',margins=True)
tabla_pagos_children

debt,Con deuda,Sin deuda,All
group_children,,,
2-3 hijos,647,6336,6983
4-5 hijos,31,349,380
Sin hijos,1063,13028,14091
All,1741,19713,21454


In [94]:
# Calculamos la tasa de incumplimiento en función del número de hijos
tabla_pagos_children['Incumplimiento']=(tabla_pagos_children['Con deuda']/tabla_pagos_children['All'])*100
tabla_pagos_children

debt,Con deuda,Sin deuda,All,Incumplimiento
group_children,,,,
2-3 hijos,647,6336,6983,9.265359
4-5 hijos,31,349,380,8.157895
Sin hijos,1063,13028,14091,7.543822
All,1741,19713,21454,8.115037


**Conclusión**

De la tabla anterior podemos notar que **las personas sin hijos son las que al parecer tienen la menor tasa de incumplimiento** en el pago de los préstamos. Así, al parecer una persona sin hijos es más probable que pague su préstamo a tiempo.

[Contenido](#back)
<a id='hypotheses_2'></a>
### ¿Existe una correlación entre la situación familiar y el pago a tiempo?

In [95]:
# Comprobamos los datos del estado familiar y los pagos a tiempo
tabla_pagos_family=datos_credito.pivot_table(index='family_status',columns='debt',values='total_income',aggfunc='count',margins=True)
tabla_pagos_family

debt,Con deuda,Sin deuda,All
family_status,,,
civil partnership,388,3763,4151
divorced,85,1110,1195
married,931,11408,12339
unmarried,274,2536,2810
widow / widower,63,896,959
All,1741,19713,21454


In [96]:
# Calculamos la tasa de incumplimiento basada en el estado familiar
tabla_pagos_family['Incumplimiento']=(tabla_pagos_family['Con deuda']/tabla_pagos_family['All'])*100
tabla_pagos_family

debt,Con deuda,Sin deuda,All,Incumplimiento
family_status,,,,
civil partnership,388,3763,4151,9.347145
divorced,85,1110,1195,7.112971
married,931,11408,12339,7.545182
unmarried,274,2536,2810,9.750890
widow / widower,63,896,959,6.569343
All,1741,19713,21454,8.115037


**Conclusión**

De la tabla anterior podemos notar que las personas solteras **(unmarried)** o en unión civil **(civil partnership)** tienen las mayores tasas de incumplimiento en el pago de su préstamo.

[Contenido](#back)
<a id='hypotheses_3'></a>
### ¿Existe una correlación entre el nivel de ingresos y el pago a tiempo?

In [97]:
# Comprobamos los datos del nivel de ingresos y los pagos a tiempo
tabla_pagos_income_type=datos_credito.pivot_table(index='group_total_income',columns='debt',values='total_income',aggfunc='count',margins=True)
tabla_pagos_income_type

debt,Con deuda,Sin deuda,All
group_total_income,,,
"100,001-150,000",4.0,67.0,71
"150,001-200,000",1.0,16.0,17
"200,001-250,000",NaN,5.0,5
"250,001-300,000",NaN,4.0,4
"50,001-100,000",86.0,1136.0,1222
"mayor de 350,000",1.0,1.0,2
"menor a 50,000",1649.0,18484.0,20133
All,1741.0,19713.0,21454


In [98]:
# Calculamos la tasa de incumplimiento basada en el nivel de ingresos
tabla_pagos_income_type['Incumplimiento']=(tabla_pagos_income_type['Con deuda']/tabla_pagos_income_type['All'])*100
tabla_pagos_income_type

debt,Con deuda,Sin deuda,All,Incumplimiento
group_total_income,,,,
"100,001-150,000",4.0,67.0,71,5.633803
"150,001-200,000",1.0,16.0,17,5.882353
"200,001-250,000",NaN,5.0,5,NaN
"250,001-300,000",NaN,4.0,4,NaN
"50,001-100,000",86.0,1136.0,1222,7.037643
"mayor de 350,000",1.0,1.0,2,50.000000
"menor a 50,000",1649.0,18484.0,20133,8.190533
All,1741.0,19713.0,21454,8.115037


**Conclusión**

Observamos que en general, las personas con un nivel bajo de ingresos, menores a 50000, presentan una mayor tasa de incumplimiento. En el caso de personas con ingreso mayor a 350000 no podemos asegurar algo, respecto al pago de un prestamo, dado que únicamente contamos con dos registros.

[Contenido](#back)
<a id='hypotheses_4'></a>
### ¿Cómo afecta el propósito del crédito a la tasa de incumplimiento?

In [99]:
# Consultamos los porcentajes de tasa de incumplimiento para cada propósito del crédito
tabla_pagos_purpose=datos_credito.pivot_table(index='group_purpose',columns='debt',values='total_income',aggfunc='count',margins=True)
tabla_pagos_purpose

debt,Con deuda,Sin deuda,All
group_purpose,,,
casarse,186,2138,2324
compra carro,403,3903,4306
inversión inmobiliaria,864,10851,11715
preparación académica,288,2821,3109
All,1741,19713,21454


In [100]:
# Calculamos la tasa de incumplimiento basada en el nivel de ingresos
tabla_pagos_purpose['Incumplimiento']=(tabla_pagos_purpose['Con deuda']/tabla_pagos_purpose['All'])*100
tabla_pagos_purpose

debt,Con deuda,Sin deuda,All,Incumplimiento
group_purpose,,,,
casarse,186,2138,2324,8.003442
compra carro,403,3903,4306,9.359034
inversión inmobiliaria,864,10851,11715,7.375160
preparación académica,288,2821,3109,9.263429
All,1741,19713,21454,8.115037


**Conclusión**

De la tabla anterior, **podemos observar que pedir un préstamo para comprar un carro o prepararse académicamente tienen las mayores tasas de incumplimiento**. Esto pueda deberse a que la compra de un carro se considera un gasto, a menos que se ocupe para generar ingresos. Y en el caso de la preparación académica, posiblemente porque el tiempo para el retorno de esa inversión puede llevar mucho tiempo, entre el tiempo de estudio y empezar a laborar.

# Conclusión general <a id='conclusion'></a>

La variable `'days_employed'`presentó demasiados problemas tales como: valores atípicos, valores ausentes y valores problématicos. Lo cual, la lleva a ser una variable que no es muy apta para usar en la estimación de valores ausentes presentes en otras variables. La creación de grupos de categorías en algunas variables nos ayudo a realizar la estimación de valores ausentes en otras categorías. Quizás haga falta probar y comparar el comportamiento de los valores ausentes reemplazados usando alguna otra medida, es decir; si usamos la media en un principio para reemplazar algún valor ausente, habría que probar que tanto afecta en el resultado usar la mediana, y de manera viceversa. También podríamos probar hacer la estimación de estos valores ausentes empleando algún método de Machine Learning o regresión lineal.

En general podemos decir que:

* **Las personas sin hijos son las que al parecer tienen la menor tasa de incumplimiento** en el pago de los préstamos. Así, al parecer una persona sin hijos es más probable que pague su préstamo a tiempo.
* **Las personas solteras (unmarried) o en unión civil (civil partnership) tienen las mayores tasas de incumplimiento** en el pago de su préstamo.
* **Las personas con un nivel bajo de ingresos, menores a 50000, presentan una mayor tasa de incumplimiento.** En el caso de personas con ingreso mayor a 350000 no podemos asegurar algo respecto al pago de un prestamo dado que únicamente contamos con dos registros.
* **Pedir un préstamo para comprar un carro o prepararse académicamente tienen las mayores tasas de incumplimiento**.

[Contenido](#back)